## SARS-CoV-2 Genome Mutator

In [1]:
import io
import os
import sys
import pandas as pd
import numpy as np

from webapp import *
from webapp1 import mut_generate, translate

from Levenshtein import distance, ratio, hamming
import ipywidgets as widgets
from IPython.display import Javascript, display

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 8)

In [2]:
MODEL_PATH = "data/ncbi_02/model_gru_bi_256"
encoder_model, decoder_model = load_model(model_path=MODEL_PATH)

Loaded model from data/ncbi_02/model_gru_bi_256.


In [3]:
def run_next_cell(ev=None):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [4]:
file_upload = widgets.FileUpload(accept='.txt', multiple=False, )
display(file_upload)

def on_upload(change):
    run_next_cell()

file_upload.observe(on_upload, names='_counter')

FileUpload(value={}, accept='.txt', description='Upload')

<IPython.core.display.Javascript object>

In [10]:
if len(file_upload.value) > 0:
    file = list(file_upload.value.values())[0]
    data = pd.read_csv(io.BytesIO(file['content']), names=['seq'])
    data['seq_len'] = data.seq.str.len()
else:
    data = None
run_next_cell()
data

<IPython.core.display.Javascript object>

,seq,seq_len
0,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,3822
1,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,3822
2,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,3822
3,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,3822
4,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...,3822


In [12]:
if data is not None:

    btn_mutate = widgets.Button(description='Mutate', icon='check')
    gen_slider = widgets.IntSlider(value=1, min=1, max=1000, step=1, 
                                   description='Generations', 
                                   layout=widgets.Layout(width='600px'))

    layout = widgets.HBox([
        gen_slider,
        btn_mutate
    ])
    display(layout)

    def on_mutate(change):
        run_next_cell()

    btn_mutate.on_click(on_mutate)

In [7]:
if data is not None:

    gen_label = widgets.Label(value='0/0')
    gen_progress = widgets.IntProgress(min=1, max=SEQ_LEN)

    layout = widgets.HBox([ 
        gen_progress, 
        gen_label 
    ])
    display(layout)

    if 'mut' in data.columns:
        mut_vector = data.mut
    else:
        mut_vector = data.seq
        
    seq_vector = encode_seq(mut_vector)

    if 'gen' in data.columns:
        gen_vector = data.gen
    else:
        gen_vector = np.zeros((len(seq_vector)), dtype=int)
        
    for i in range(gen_slider.value):

        gen_label.value = f'{i + 1}/{gen_slider.value}'
        seq_vector = mut_generate(a=seq_vector)
        
        for i in range(len(seq_vector)):
            

    run_next_cell()

In [8]:
if data is not None:

    data['mut'] = decode_seq(seq_vector)
    data['mut_len'] = data.mut.str.len()

    data['lev_distance'] = [ distance(x,y) for x,y in zip(data.seq, data.mut) ]
    data['lev_ratio'] = [ ratio(x,y) for x,y in zip(data.seq, data.mut) ]
    data['hamming'] = [ hamming(x,y) for x,y in zip(data.seq, data.mut) ]

    run_next_cell()
    
data

In [9]:
if data is not None:
    freq = None
    for i in range(len(data)):
        d = np.where(~(np.array(list(data.seq[i])) == np.array(list(data.mut[i]))))[0]
        freq = d if freq is None else np.append(freq, d)

    plt.hist(freq, bins=250)
    plt.show()